<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/microsoft_phi_2_zero_shot_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U accelerate bitsandbytes bert-score

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
model_name = "microsoft/phi-2"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


2024-02-19 18:21:46.076205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 18:21:46.076308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 18:21:46.252379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
prompt = "Instruct:"+"What is the heaviest element?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(sequence[0]['generated_text'])

Instruct:What is the heaviest element?
Output: The heaviest element is Oganesson.



In [ ]:
sequence

[{'generated_text': 'Instruct:What is the chemical formula of water? Here is some info: It includes H2 and O2 molecules\nOutput: The chemical formula of water is H2O.\n'}]

In [ ]:
prompt = "Instruct:"+"What is the gravity on mars?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)
print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct:What is the gravity on mars?
Output: The gravity on Mars is 3.711 m/s^2.



In [ ]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [ ]:
prompt = "Instruct:"+"Who invented mirror telescope?"+"\nOutput:"

print(Sequence(prompt)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct:Who invented mirror telescope?
Output: The first recorded instance of a mirror telescope was in 1608 by Dutch spectacle maker Hans Lippershey.



In [ ]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [ ]:
dataset_validation = load_dataset('sciq',split='validation[80%:]') #last %20 of validation data

In [ ]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [ ]:
answers = []
for question in dataset_validation["question"]:
    prompt = "Instruct:"+question+"\nOutput:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = txt.split("\nOutput: ")
    answer = question_and_answer[1]
    answers.append(answer)

In [ ]:
from bert_score import score

# Assuming `answers` is your list of generated answers and `actual_answers` is the list of reference answers
# Both should be lists of strings

# Calculate BertScore
P, R, F1 = score(answers, dataset_validation["correct_answer"], lang='en', verbose=True)

# `P`, `R`, and `F1` are returned as tensors, so you might want to convert them to Python lists or take their mean
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print(f"Precision: {mean_precision:.3f}")
print(f"Recall: {mean_recall:.3f}")
print(f"F1: {mean_f1:.3f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 0.47 seconds, 421.66 sentences/sec
Precision: 0.796
Recall: 0.795
F1: 0.795
